#### Don't forget to delete the hdmi_out and hdmi_in when finished

# Motion Blur Filter Example
In this notebook, we will demonstrate how to use the motion blur filter. This filter shows that partially reconfigurable modules can use Xilinx IP cores. This filter blurs the video feed horizontally. The length of the blur is determined by a register in the module. This register is controlled by a python slide widget.
<img src="data/motion.jpg"/>
This filter works by adding up the RBG values of the pixels to the left of the pixel being displayed and then dividing by the number of pixels. The number of pixels to blur is determined by a register. A Xilinx divider core is needed to operate the dividing. This is because the numerator and denominator of the division are variables.


#### 1. Download base overlay to the board
Ensure that the camera is not connected to the board. Run the following script to provide the PYNQ with its base overlay.

In [1]:
from pynq.drivers.video import HDMI
from pynq import Bitstream_Part
from pynq.board import Register
from pynq import Overlay

Overlay("demo.bit").download()

#### 2. Connect camera
Physically connect the camera to the HDMI-in port of the PYNQ. Run the following code to instruct the PYNQ to capture the video from the camera and to begin streaming video to your monitor (connected to the HDMI-out port).

In [2]:
hdmi_in = HDMI('in')
hdmi_out = HDMI('out', frame_list=hdmi_in.frame_list)
hdmi_out.mode(2)
hdmi_out.start()
hdmi_in.start()

#### 3. Program board
Run the following script to download the Motion Blur Filter to the PYNQ.

In [3]:
Bitstream_Part("motion_p.bit").download()

#### 4. Create a user interface
We will communicate with the filter using a nice user interface. Run the following code to activate that interface.

In [5]:
import ipywidgets as widgets

R0 =Register(0)

R0.write(255)

R0_s = widgets.IntSlider(
    value=255,
    min=0,
    max=511,
    step=1,
    description='Blur:',
    disabled=False,
    continuous_update=True,
    orientation='vertical',
    readout=True,
    readout_format='i',
    slider_color='red'
)

def update_r0(*args):
    R0.write(R0_s.value)
R0_s.observe(update_r0, 'value')

widgets.HBox([R0_s])

#### 5. Exploration
Move the slider above to change the lenght of the blur. When the slider is set to zero there is no blur. Notice how quickly the filter responds to the movement of the slider.

#### 6. Clean up
When you are done with the filter, run the following code to stop the video stream

In [6]:
hdmi_out.stop()
hdmi_in.stop()
del hdmi_out
del hdmi_in